In [ ]:
!git clone https://github.com/syrgkanislab/npiv_functionals.git

In [ ]:
%cd npiv_functionals

In [ ]:
!python setup.py install

In [ ]:
import os
import sys
sys.path.append(os.path.abspath('.'))

In [ ]:
import warnings
warnings.simplefilter('ignore')
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from mliv.dgps import get_data, get_tau_fn, fn_dict
from mliv.neuralnet.utilities import mean_ci
from mliv.neuralnet import AGMMEarlyStop as AGMM
from mliv.neuralnet.moments import avg_small_diff
from sklearn.ensemble import RandomForestRegressor
import joblib
from joblib import Parallel, delayed
from mliv.cct.mc2 import MC2
from mliv.rkhs import ApproxRKHSIVCV
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.kernel_approximation import Nystroem
from sklearn.pipeline import FeatureUnion, Pipeline
import scipy
import pandas as pd

In [ ]:
# average finite difference moment
def moment_fn(x, test_fn):
    epsilon = 0.1
    t1 = np.hstack([x[:, [0]] + epsilon, x[:, 1:]])
    t0 = np.hstack([x[:, [0]] - epsilon, x[:, 1:]])
    return (test_fn(t1) - test_fn(t0)) / (2 * epsilon)

In [ ]:
def moment_evals(x):
    epsilon = 0.1
    t1 = np.hstack([x[:, [0]] + epsilon, x[:, 1:]])
    t0 = np.hstack([x[:, [0]] - epsilon, x[:, 1:]])
    return t0, t1

In [ ]:
it = 0
n = 5000
mc2_gen = MC2(n, 100, None, dimension=10, corr=0.5)
npvec, *_ = mc2_gen.data(it)
Z, X, Y = npvec['instrument'], npvec['endogenous'], npvec['response']
n_z = Z.shape[1]
n_x = X.shape[1]

In [ ]:
Z_train, Z_val, X_train, X_val, Y_train, Y_val = train_test_split(
        Z, X, Y, test_size=.5, shuffle=True)

ztrans = Nystroem(n_components=100)
xtrans = Nystroem(n_components=100)
# ztrans = PolynomialFeatures(degree=2)
# xtrans = PolynomialFeatures(degree=2)
# ztrans = FeatureUnion([('poly', PolynomialFeatures(degree=2)), ('nys', Nystroem(n_components=10))])
# xtrans = FeatureUnion([('poly', PolynomialFeatures(degree=2)), ('nys', Nystroem(n_components=10))])
ztrans = Pipeline([('trans', ztrans), ('scale', StandardScaler())])
xtrans = Pipeline([('trans', xtrans), ('scale', StandardScaler())])
Psi = ztrans.fit_transform(Z_train)
xtrans.fit(np.vstack((X_train,) + moment_evals(X_train)))
Phi = xtrans.transform(X_train)
mPhi = moment_fn(X_train, xtrans.transform)

In [ ]:
CovPsi = Psi.T @ Psi
CovPhiPsi = Phi.T @ Psi
Phival = xtrans.transform(X_val)
Psival = ztrans.transform(Z_val)
mPhival = moment_fn(X_val, xtrans.transform)
moment_val = np.mean(mPhival, axis=0)

best_violation = np.inf
for alpha in np.logspace(-6, 1, 5):
    regCov = scipy.linalg.pinv(CovPsi + alpha * n * np.eye(Psi.shape[1]))
    Sigma = CovPhiPsi @ regCov @ CovPsi @ regCov @  CovPhiPsi.T
    for beta in np.logspace(-6, 1, 5):
        xi = scipy.linalg.pinv(Sigma + beta * n * np.eye(Phi.shape[1])) @ np.sum(mPhi, axis=0)
        for gamma in np.logspace(-6, 1, 5):
            qparam = scipy.linalg.pinv(CovPsi + gamma * n * np.eye(Psi.shape[1])) @ CovPhiPsi.T @ xi

            representer_val = np.mean((Psival @ qparam).reshape(-1, 1) * Phival, axis=0)
            violation = np.linalg.norm(moment_val - representer_val, ord=2)
            if violation <= best_violation:
                best_alpha = alpha
                best_beta = beta
                best_gamma = gamma
                best_violation = violation

In [ ]:
alpha = best_alpha
beta = best_beta
gamma = best_gamma
regCov = scipy.linalg.pinv(CovPsi + alpha * n * np.eye(Psi.shape[1]))
Sigma = CovPhiPsi @ regCov @ CovPsi @ regCov @  CovPhiPsi.T
xi = scipy.linalg.pinv(Sigma + beta * n * np.eye(Phi.shape[1])) @ np.sum(mPhi, axis=0)
qparam = scipy.linalg.pinv(CovPsi + gamma * n * np.eye(Psi.shape[1])) @ CovPhiPsi.T @ xi

In [ ]:
best_alpha, best_beta, best_gamma, best_violation

In [ ]:
agmm = ApproxRKHSIVCV(n_components=200)
agmm.fit(Z_train, X_train, Y_train)

In [ ]:
direct = moment_fn(X_val, agmm.predict).flatten()
residual = (Y_val - agmm.predict(X_val)).flatten()
qvalues = Psival @ qparam
pseudo = direct + qvalues * residual

reg = mean_ci(direct)
dr = mean_ci(pseudo)
ipw = mean_ci(qvalues * Y_val.flatten())
reg, ipw, dr

In [ ]:
xivalues = xtrans.transform(X_val) @ xi
coef = np.mean(qvalues * residual) / np.mean(qvalues * xivalues)
pseudo_tmle = direct + coef * (mPhival @ xi)
pseudo_tmle += qvalues * (residual - coef * xivalues)
tmle = mean_ci(pseudo_tmle)
tmle

In [ ]:
from sklearn.model_selection import KFold

def exp(it, n, dim, corr, fname='cct', iv_strength=None, endogeneity_strength=None):
    np.random.seed(it)
    if fname == 'cct':
        mc2_gen = MC2(n, 100, None, dimension=dim, corr=corr)
        npvec, *_ = mc2_gen.data(it)
        Z, X, Y = npvec['instrument'], npvec['endogenous'], npvec['response']
    else:
        Z, X, Y, _ = get_data(n, 1, iv_strength, get_tau_fn(fn_dict[fname]), 5, endogeneity_strength=endogeneity_strength)

    direct = np.zeros(n)
    residual = np.zeros(n)
    qvalues = np.zeros(n)
    xivalues = np.zeros(n)
    mxivalues = np.zeros(n)

    for train, test in KFold(n_splits=5, shuffle=True).split(Z):
        Z_train, Z_val, X_train, X_val, Y_train, Y_val = Z[train], Z[test], X[train], X[test], Y[train], Y[test]

        ztrans = Nystroem(n_components=200)
        xtrans = Nystroem(n_components=200)
        ztrans = Pipeline([('trans', ztrans), ('scale', StandardScaler())])
        xtrans = Pipeline([('trans', xtrans), ('scale', StandardScaler())])

        Psi = ztrans.fit_transform(Z_train)
        xtrans.fit(np.vstack((X_train,) + moment_evals(X_train)))
        Phi = xtrans.transform(X_train)
        mPhi = moment_fn(X_train, xtrans.transform)

        CovPsi = Psi.T @ Psi
        CovPhiPsi = Phi.T @ Psi
        Phival = xtrans.transform(X_val)
        Psival = ztrans.transform(Z_val)
        mPhival = moment_fn(X_val, xtrans.transform)
        moment_val = np.mean(mPhival, axis=0)

        best_violation = np.inf
        for alpha in np.logspace(-7, 1, 10):
            regCov = scipy.linalg.inv(CovPsi + alpha * n * np.eye(Psi.shape[1]))
            Sigma = CovPhiPsi @ regCov @ CovPsi @ regCov @  CovPhiPsi.T
            for beta in np.logspace(-7, 1, 10):
                xi = scipy.linalg.inv(Sigma + beta * n * np.eye(Phi.shape[1])) @ np.sum(mPhi, axis=0)
                for gamma in np.logspace(-7, 1, 10):
                    qparam = scipy.linalg.inv(CovPsi + gamma * n * np.eye(Psi.shape[1])) @ CovPhiPsi.T @ xi

                    # calculating the violation in the riesz representation property for each feature
                    #  E[m(W; phi)] = E[q(Z) * phi(X)]
                    # for every feature phi.
                    representer_val = np.mean((Psival @ qparam).reshape(-1, 1) * Phival, axis=0)
                    violation = np.linalg.norm(moment_val - representer_val, ord=2)
                    if violation <= best_violation:
                        best_alpha = alpha
                        best_beta = beta
                        best_gamma = gamma
                        best_violation = violation

        alpha = best_alpha
        beta = best_beta
        gamma = best_gamma
        regCov = scipy.linalg.inv(CovPsi + alpha * n * np.eye(Psi.shape[1]))
        Sigma = CovPhiPsi @ regCov @ CovPsi @ regCov @  CovPhiPsi.T
        xi = scipy.linalg.inv(Sigma + beta * n * np.eye(Phi.shape[1])) @ np.sum(mPhi, axis=0)
        qparam = scipy.linalg.inv(CovPsi + gamma * n * np.eye(Psi.shape[1])) @ CovPhiPsi.T @ xi

        agmm = ApproxRKHSIVCV(n_components=200)
        agmm.fit(Z_train, X_train, Y_train)

        direct[test] = moment_fn(X_val, agmm.predict).flatten()
        residual[test] = (Y_val - agmm.predict(X_val)).flatten()
        qvalues[test] = Psival @ qparam
        xivalues[test] = Phival @ xi
        mxivalues[test] = mPhival @ xi

    pseudo = direct + qvalues * residual

    reg = mean_ci(direct)
    dr = mean_ci(pseudo)
    ipw = mean_ci(qvalues * Y.flatten())

    coef = np.mean(qvalues * residual) / np.mean(qvalues * xivalues)
    pseudo_tmle = direct + coef * mxivalues
    pseudo_tmle += qvalues * (residual - coef * xivalues)
    tmle = mean_ci(pseudo_tmle)

    return dr, tmle, ipw, reg

In [ ]:
def get_result_dict(results, true, alpha=0.95):
    df = {}
    for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
        if method == 'ipw':
            continue
        data = np.array([r[it] for r in results])
        confidence = .95
        se = (data[:, 2] - data[:, 0]) / scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
        if method in ['dr', 'tmle']:
            confidence = 0.95
            data[:, 1] = data[:, 0] - se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
            data[:, 2] = data[:, 0] + se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
            cov95 = f'{100*np.mean((data[:, 1] <= true) & (true <= data[:, 2])):.0f}'
            confidence = 0.99
            data[:, 1] = data[:, 0] - se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
            data[:, 2] = data[:, 0] + se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
            cov99 = f'{100*np.mean((data[:, 1] <= true) & (true <= data[:, 2])):.0f}'
        else:
            cov = 'NA'
        df[method] =  {'cov95': cov95, 'cov99': cov99,
                'rmse': f'{np.sqrt(np.mean((data[:, 0] - true)**2)):.3f}',
                'bias': f'{np.abs(np.mean((data[:, 0] - true))):.3f}'}
    return df

In [ ]:
true = 1.0

for n in [1000, 5000]:
    for n_x in [0, 5, 10]:
        for corr in [0.0, 0.5]:
            if n_x == 0 and corr == 0.5:
                continue
            print(n, n_x, corr)
            results = Parallel(n_jobs=-1, verbose=3)(delayed(exp)(it, n, n_x, corr)
                                                            for it in range(1000))
            joblib.dump(results, f'rkhs_cct_n_{n}_n_x_{n_x}_corr_{corr}.jbl')
            df = pd.DataFrame(get_result_dict(results, true))
            display(df)

In [ ]:
true = 1.0
res = {}
for n_x in [0, 5, 10]:
    res[f'${n_x}$'] = {}
    for n in [1000, 5000]:
        res[f'${n_x}$'][f'${n}$'] = {}
        for corr in [0.0, 0.5]:
            if n_x == 0 and corr == 0.5:
                continue
            results = joblib.load(f'rkhs_cct_n_{n}_n_x_{n_x}_corr_{corr}.jbl')
            res[f'${n_x}$'][f'${n}$'][f'${corr}$'] = pd.DataFrame(get_result_dict(results, true))
        res[f'${n_x}$'][f'${n}$'] = pd.concat(res[f'${n_x}$'][f'${n}$'], sort=False)
    res[f'${n_x}$'] = pd.concat(res[f'${n_x}$'], sort=False)
res = pd.concat(res, sort=False).unstack(level=3)
print(res.to_latex(bold_rows=True, multirow=True,
                   multicolumn=True, escape=False,
                   column_format='lll||llll|llll|llll|',
                   multicolumn_format='c|'))

In [ ]:
for fname in ['abs', '2dpoly', 'sigmoid', 'sin']:
    for iv_strength in [.2, .5]:
        for endogeneity_strength in [0.1, 0.3]:
            Z, X, Y, true_fn = get_data(
                1000000, 1, iv_strength, get_tau_fn(fn_dict[fname]), 5, endogeneity_strength=endogeneity_strength)
            true = np.mean(moment_fn(X, true_fn))
            for n in [500, 1000, 2000]:
                print(n, fname, iv_strength, endogeneity_strength, true)
                results = Parallel(n_jobs=-1, verbose=3)(delayed(exp)(it, n, None, None,
                                                                      fname=fname,
                                                                      iv_strength=iv_strength,
                                                                      endogeneity_strength=endogeneity_strength)
                                                                for it in range(1000))
                joblib.dump((true, results), f'rkhs_fname_{fname}_n_{n}_iv_strength_{iv_strength}_{endogeneity_strength}.jbl')
                df = pd.DataFrame(get_result_dict(results, true))
                display(df)

In [ ]:
def get_result_dict(results, true, alpha=0.95):
    df = {}
    for it, method in enumerate(['dr', 'tmle', 'ipw', 'direct']):
        data = np.array([r[it] for r in results])
        confidence = .95
        se = (data[:, 2] - data[:, 0]) / scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
        confidence = alpha
        data[:, 1] = data[:, 0] - se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
        data[:, 2] = data[:, 0] + se * scipy.stats.t.ppf((1 + confidence) / 2., n - 1)
        if method in ['dr', 'tmle']:
            cov = f'{100*np.mean((data[:, 1] <= true) & (true <= data[:, 2])):.0f}'
        else:
            cov = 'NA'
        df[method] =  {'cov': cov,
                'rmse': f'{np.sqrt(np.mean((data[:, 0] - true)**2)):.3f}',
                'bias': f'{np.abs(np.mean((data[:, 0] - true))):.3f}'}
    return df

In [ ]:
alpha = 0.95
res = {}
for fname in ['abs', '2dpoly', 'sigmoid', 'sin']:
    res[fname] = {}
    for n in [500, 1000, 2000]:
        res[fname][f'${n}$'] = {}
        for iv_strength in [.2, .5]:
            res[fname][f'${n}$'][f'${iv_strength}$'] = {}
            for endogeneity_strength in [0.1, 0.3]:
                true, results = joblib.load(f'rkhs_fname_{fname}_n_{n}_iv_strength_{iv_strength}_{endogeneity_strength}.jbl')
                df = pd.DataFrame(get_result_dict(results, true, alpha=alpha))
                res[fname][f'${n}$'][f'${iv_strength}$'][f'${endogeneity_strength}$'] = df
            res[fname][f'${n}$'][f'${iv_strength}$'] = pd.concat(res[fname][f'${n}$'][f'${iv_strength}$'], sort=False)
        res[fname][f'${n}$'] = pd.concat(res[fname][f'${n}$'], sort=False)
    res[fname] = pd.concat(res[fname], sort=False)
res = pd.concat(res, sort=False).unstack(level=4)
display(res)
print(res.to_latex(bold_rows=True, multirow=True,
                   multicolumn=True, escape=False,
                   column_format='llll||lll|lll|lll|lll|',
                   multicolumn_format='c|'))